In [2]:
import torch
torch.cuda.is_available()

True

In [1]:
from ultralytics import YOLO
model = YOLO('yolo11n.pt')


In [ ]:

model.train(data = r"data_for_detection\\data.yaml",epochs = 100,imgsz = 640,device = -1,batch = 16)

Searching for 1 idle GPUs with free memory >= 20.0% and free utilization >= 0.0%...
Selected idle CUDA devices [0]
Ultralytics 8.3.203  Python-3.13.3 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data_for_detection\\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=t

In [7]:
import os

# --- НАСТРОЙКИ ---
label_folders = [
    r"data_for_detection/labels/train",
    r"data_for_detection/labels/val",
    r"data_for_detection/labels/test" 
]
# Укажи индекс класса, который мы хотим оставить (в твоем случае 'plate' -> 1)
TARGET_CLASS_INDEX = 1
# -----------------

print("--- Начинаем фильтрацию файлов разметки ---")
files_processed = 0
files_deleted = 0

for folder in label_folders:
    if not os.path.exists(folder):
        print(f"Папка не найдена, пропускаю: {folder}")
        continue
        
    print(f"Обрабатываем папку: {folder}")
    
    for filename in os.listdir(folder):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder, filename)
            
            with open(file_path, 'r') as f:
                lines = f.readlines()
            
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if not parts:
                    continue
                
                class_id = int(parts[0])
                
                # Если это наш целевой класс
                if class_id == TARGET_CLASS_INDEX:
                    # Заменяем старый индекс на 0 и добавляем в новый список
                    new_line = f"0 {' '.join(parts[1:])}\n"
                    new_lines.append(new_line)
            
            # Если в файле остались нужные нам объекты
            if new_lines:
                with open(file_path, 'w') as f:
                    f.writelines(new_lines)
            # Если в файле не осталось нужных объектов
            else:
                os.remove(file_path)
                files_deleted += 1

            files_processed += 1

print("\n--- Фильтрация завершена ---")
print(f"Обработано файлов: {files_processed}")
print(f"Удалено пустых файлов разметки: {files_deleted}")
print("\nВАЖНО: Не забудьте обновить ваш data.yaml файл!")

--- Начинаем фильтрацию файлов разметки ---
Обрабатываем папку: data_for_detection/labels/train
Обрабатываем папку: data_for_detection/labels/val
Обрабатываем папку: data_for_detection/labels/test

--- Фильтрация завершена ---
Обработано файлов: 452
Удалено пустых файлов разметки: 4

ВАЖНО: Не забудьте обновить ваш data.yaml файл!


In [8]:
# delete_orphan_images_v3.py
import os

# --- НАСТРОЙКИ ---
# Укажи полные пути к папкам с изображениями и разметкой
# Важно, чтобы порядок папок соответствовал друг другу!
image_folders = [
    r"data_for_detection/images/train",
    r"data_for_detection/images/val",
    r"data_for_detection/images/test"
]
label_folders = [
    r"data_for_detection/labels/train",
    r"data_for_detection/labels/val",
    r"data_for_detection/labels/test" 
]
# -----------------

print("--- Начинаем удаление 'осиротевших' изображений ---")
files_checked = 0
files_deleted = 0

# Проходим по парам папок (images/train и labels/train, и т.д.)
for images_path, labels_path in zip(image_folders, label_folders):
    
    if not os.path.exists(images_path) or not os.path.exists(labels_path):
        print(f"ВНИМАНИЕ: Пропускаю пару, т.к. одна из папок не найдена: '{images_path}' или '{labels_path}'")
        continue
        
    print(f"Сравниваем папки: '{os.path.basename(images_path)}' и '{os.path.basename(labels_path)}'")
    
    # Создаем множество имен файлов разметки для быстрой проверки
    label_basenames = {os.path.splitext(f)[0] for f in os.listdir(labels_path)}
    
    for image_filename in os.listdir(images_path):
        files_checked += 1
        image_basename = os.path.splitext(image_filename)[0]
        
        # Если имя изображения (без расширения) отсутствует в именах разметки
        if image_basename not in label_basenames:
            image_filepath = os.path.join(images_path, image_filename)
            os.remove(image_filepath)
            files_deleted += 1
            # Раскомментируй следующую строку, если хочешь видеть каждый удаленный файл
            # print(f"  Удалено: {image_filepath}")

print("\n--- Очистка завершена ---")
print(f"Проверено файлов изображений: {files_checked}")
print(f"Удалено 'осиротевших' изображений: {files_deleted}")

--- Начинаем удаление 'осиротевших' изображений ---
Сравниваем папки: 'train' и 'train'
Сравниваем папки: 'val' и 'val'
Сравниваем папки: 'test' и 'test'

--- Очистка завершена ---
Проверено файлов изображений: 452
Удалено 'осиротевших' изображений: 4
